In [4]:
# coding: utf-8
# プロセス分散
# pip install futures

import time
from concurrent import futures
import os
from multiprocessing import Manager
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='[%(levelname)s] time:%(created).8f pid:%(process)d pn:%(processName)-10s tid:%(thread)d tn:%(threadName)-10s fn:%(funcName)-10s %(message)s',
)

'''
ここはプロセスで実行される
SHARED_VARIABLE['SOMETHING1_READY']=True であるうちは実行を続ける
'''
def do_something1():
    logging.debug("enter")
    SHARED_VARIABLE['SOMETHING1_READY']=True

    ####################
    # ループ実行
    ####################
    execute_time=0.0
    execute_clock_time=0.0
    count = 0
    while SHARED_VARIABLE['SOMETHING1_READY']:
        start_time,clock_time=time.time(),time.clock()
        SHARED_VARIABLE['SOMETHING1_VALUE'] += 1
        execute_time += time.time() - start_time
        execute_clock_time += time.clock() - clock_time
        count+=1
        logging.debug("value:%s" % (str(SHARED_VARIABLE['SOMETHING1_VALUE'])))
        time.sleep(0.1)
    logging.debug("ave time:%.8f ave clock:%.8f" % (execute_time/count,execute_clock_time/count))
    return

'''
ここはプロセスで実行される
SHARED_VARIABLE['SOMETHING2_READY']=True であるうちは実行を続ける
'''
def do_something2():
    logging.debug("enter")
    SHARED_VARIABLE['SOMETHING2_READY']=True

    ####################
    # ループ実行
    ####################
    execute_time=0.0
    execute_clock_time=0.0
    count = 0
    while SHARED_VARIABLE['SOMETHING2_READY']:
        start_time,clock_time=time.time(),time.clock()
        SHARED_VARIABLE['SOMETHING2_VALUE'] = SHARED_VARIABLE['SOMETHING1_VALUE'] << 1 # SHARED_VARIABLE['SOMETHING1_VALUE'] * 2
        execute_time += time.time() - start_time
        execute_clock_time += time.clock() - clock_time
        count+=1
        logging.debug("value:%s" % (str(SHARED_VARIABLE['SOMETHING2_VALUE'])))
        time.sleep(0.1)
    logging.debug("ave time:%.8f ave clock:%.8f" % (execute_time/count,execute_clock_time/count))
    return

'''
ここはプロセスで実行される
RUNNING_SEC後に各プロセスが停止するようにBOOL型変数を書き換える
'''
def do_stop():
    logging.debug("enter")

    ####################
    # ループ実行
    ####################
    start_time = time.time()
    RUNNING_SEC = 10
    time.sleep(RUNNING_SEC)
    SHARED_VARIABLE['SOMETHING1_READY']=False
    SHARED_VARIABLE['SOMETHING2_READY']=False
        
    return

'''
process pattern
'''
SHARED_VARIABLE=Manager().dict()
SHARED_VARIABLE['SOMETHING1_READY']=False
SHARED_VARIABLE['SOMETHING2_READY']=False
SHARED_VARIABLE['SOMETHING1_VALUE']=0
SHARED_VARIABLE['SOMETHING2_VALUE']=0

'''
プロセスによる実行関数の振り分け定義
'''
PROCESS_LIST=['do_something1','do_something2','do_stop']
def do_process(target):

    if target == 'do_something1':
        do_something1()
        return "end do_something1"
    if target == 'do_something2':
        do_something2()
        return "end do_something2"
    if target == 'do_stop':
        do_stop()
        return "end do_stop"

'''
メイン処理を行う部分
・メインスレッド（ここ）
・スレッド1(concurrent.futures)
・スレッド2(concurrent.futures)
・制御スレッド(concurrent.futures)
'''
def do_main():
    try:
        with futures.ProcessPoolExecutor(max_workers=len(PROCESS_LIST)) as executer:
            mappings = {executer.submit(do_process, pname): pname for pname in PROCESS_LIST}
            for i in futures.as_completed(mappings):
                target = mappings[i]
                result = i.result()
                print(result)

    except Exception as e:
        print('error! executer failed.')
        print(str(e))
    finally:
        print("executer end")

    return

do_main()








[DEBUG] time:1496896709.27411509 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 enter
[DEBUG] time:1496896709.28197694 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 enter
[DEBUG] time:1496896709.29643798 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:1
[DEBUG] time:1496896709.30200911 pid:836 pn:Process-16 tid:1995620352 tn:MainThread fn:do_stop    enter
[DEBUG] time:1496896709.29232693 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:0
[DEBUG] time:1496896709.40522599 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:2
[DEBUG] time:1496896709.41277003 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:4
[DEBUG] time:1496896709.51073098 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:3
[DEBUG] time:1496896709.51727200 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:6
[DEBUG] time:1496896709.6167

[DEBUG] time:1496896713.13742709 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:37
[DEBUG] time:1496896713.14828706 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:74
[DEBUG] time:1496896713.24710107 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:38
[DEBUG] time:1496896713.25620389 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:76
[DEBUG] time:1496896713.35518694 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:39
[DEBUG] time:1496896713.36417103 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:78
[DEBUG] time:1496896713.46479297 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:40
[DEBUG] time:1496896713.47210693 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:80
[DEBUG] time:1496896713.57449198 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:41
[DEBUG] ti

[DEBUG] time:1496896717.10941696 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:74
[DEBUG] time:1496896717.19666910 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:148
[DEBUG] time:1496896717.21474600 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:75
[DEBUG] time:1496896717.30108905 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:150
[DEBUG] time:1496896717.32012010 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:76
[DEBUG] time:1496896717.40545893 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:152
[DEBUG] time:1496896717.42542601 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:77
[DEBUG] time:1496896717.50984097 pid:834 pn:Process-15 tid:1995620352 tn:MainThread fn:do_something2 value:154
[DEBUG] time:1496896717.53069711 pid:833 pn:Process-14 tid:1995620352 tn:MainThread fn:do_something1 value:78
[DEBUG

end do_stop
end do_something2
end do_something1
executer end
